<a href="https://colab.research.google.com/github/Aidakazemi/BUS650/blob/main/BUSI650_Week6_TimeSeries_ABTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 📘 BUSI650 — Week 6 : Time Series Analysis & A/B Testing

---

## 🎯 Learning Objectives
By the end of this session, you will be able to:
- Understand what time series data is and how to interpret it.  
- Apply forecasting methods (naïve, moving average, exponential smoothing, regression trend).  
- Understand and apply A/B testing for business decisions.  

---

##  Warm-Up & Recap
Last week, we learned about **hypothesis testing** and **regression models** — how to predict one variable using another.

Today, we’ll move one step forward:  
👉 How to **predict the future** using **past data** — that’s called *time series forecasting*.



# ⏳ Part 1 — Time Series Basics

A **time series** is data collected over time — daily, weekly, monthly, quarterly.

In business, examples include:
- Monthly sales revenue  
- Daily website visitors  
- Quarterly profit  
- Weekly employee hours worked  
- Look at this time series data of flow of a day in life at https://flowingdata.com/2015/12/15/a-day-in-the-life-of-americans/

---



### 📊 Example: Monthly Coffee Sales (Café Case)

In [ ]:

import pandas as pd, numpy as np, matplotlib.pyplot as plt

# Generate simple time series dataset
np.random.seed(42)
dates = pd.date_range("2023-01-01", periods=24, freq="M")
sales = [200 + i*5 + np.random.randint(-10,10) for i in range(24)]  # upward trend
df = pd.DataFrame({"Month": dates, "Sales": sales})



# Plot the time series
plt.figure(figsize=(10,4))
plt.plot(df["Month"], df["Sales"], marker="o", linestyle="-")
plt.title("☕ Café Monthly Coffee Sales (2023–2024)")
plt.xlabel("Month")
plt.ylabel("Sales ($ in hundreds)")
plt.grid(True)
plt.show()


In [ ]:
# Print the data of sales and months
print(df[["Month", "Sales"]])

        Month  Sales
0  2023-01-31    196
1  2023-02-28    214
2  2023-03-31    214
3  2023-04-30    215
4  2023-05-31    217
5  2023-06-30    221
6  2023-07-31    238
7  2023-08-31    235
8  2023-09-30    240
9  2023-10-31    238
10 2023-11-30    247
11 2023-12-31    247
12 2024-01-31    251
13 2024-02-29    266
14 2024-03-31    265
15 2024-04-30    266
16 2024-05-31    270
17 2024-06-30    286
18 2024-07-31    291
19 2024-08-31    301
20 2024-09-30    299
21 2024-10-31    310
22 2024-11-30    314
23 2024-12-31    319



**Story to Tell:**
> The café wants to know how many cups of coffee it will likely sell next month (Month 25),  
> so it can plan for beans, milk, and staff scheduling.


# 🔢 Part 2 — Time Series Forecasting Methods


## 1️⃣ Naïve Forecast — "Tomorrow is Like Today"
**Concept:**  
Assumes the next month’s sales = this month’s sales.

$$
\hat{Y}_{t+1} = Y_t
$$


In [5]:

# Naive forecast
naive_forecast = df["Sales"].iloc[-1]
print("Naïve forecast for next month:", naive_forecast)


Naïve forecast for next month: 319


> Simple baseline: assumes next month = last month.


## 2️⃣ Moving Average — "Average of Last Few Months"
**Concept:** Smooths random ups and downs by averaging the most recent *n* months.

$$
\hat{Y}_{t+1} = \frac{Y_t + Y_{t-1} + \dots + Y_{t-n+1}}{n}
$$


In [6]:

n = 3
moving_avg_forecast = df["Sales"].tail(n).mean()
print("3-Month Moving Average Forecast:", round(moving_avg_forecast, 2))


3-Month Moving Average Forecast: 314.33


> Averages last 3 months for a smoother trend.


## 3️⃣ Weighted Moving Average — "More Weight to Recent Months"
**Concept:** More recent data is more relevant.

$$
\hat{Y}_{t+1} = w_1Y_t + w_2Y_{t-1} + w_3Y_{t-2}, \quad w_1+w_2+w_3=1
$$


In [8]:

weights = [0.5, 0.3, 0.2]
recent = df["Sales"].tail(3).to_numpy()[::-1]
weighted_forecast = np.sum(weights * recent)
print("Weighted Moving Average Forecast:", round(weighted_forecast, 2))


Weighted Moving Average Forecast: 315.7


> Gives more weight to recent data — captures recency.


## 4️⃣ Regression Trend — "Predict Based on Growth Pattern"
**Concept:** Fit a linear trend model to estimate consistent growth.

$$
Y_t = a + b t + \varepsilon_t
$$


In [9]:

from sklearn.linear_model import LinearRegression

df["t"] = np.arange(1, len(df)+1)
X = df[["t"]]; y = df["Sales"]
model = LinearRegression().fit(X, y)
a, b = model.intercept_, model.coef_[0]
t_next = 25
regression_forecast = a + b * t_next

print(f"Regression Equation: Sales = {a:.2f} + {b:.2f} * t")
print("Regression Forecast for next month:", round(regression_forecast, 2))


Regression Equation: Sales = 193.78 + 5.03 * t
Regression Forecast for next month: 319.56


In [10]:

print("Naïve:", naive_forecast)
print("Moving Average:", round(moving_avg_forecast,2))
print("Weighted MA:", round(weighted_forecast,2))
print("Regression:", round(regression_forecast,2))


Naïve: 319
Moving Average: 314.33
Weighted MA: 315.7
Regression: 319.56



**Discussion:**
> Which forecast seems most realistic given the trend?  
> If you were the café manager, which would you rely on?



# 🎯 Part 3 — A/B Testing (Experiments)

A/B testing compares two versions — A (control) and B (new version) — to see which performs better.



### 📈 Example: Email Campaign
A company tests two subject lines for email campaign and we want to know: Is version B significantly better?
| Group | Open Rate (%) |
|--------|---------------|
| A | 25 |
| B | 32 |

**Hypotheses:**
- $H_0$: mean(A) = mean(B)

- $H_1$: mean(B) > mean(A)


In [ ]:

import scipy.stats as stats

A = np.random.binomial(1, 0.25, 200)
B = np.random.binomial(1, 0.32, 200)

t, p = stats.ttest_ind(B, A, equal_var=False)
print(f"t = {t:.2f}, one-tailed p = {p/2:.4f}")



If *p < 0.05*, Version B performs significantly better.

> Example: “Buy One Get One Free” (A) vs. “Free Muffin with Coffee” (B).  
> If p < 0.05 → B wins; send that campaign!



# 🔚 Wrap-Up

| Concept | Purpose |
|----------|----------|
| Time Series | Understand how data changes over time |
| Forecasting | Predict future using past patterns |
| A/B Testing | Evaluate two strategies to guide decisions |

---

## 🏠 Homework
Find a dataset with a time column (e.g., sales, traffic).  
1. Plot the trend.  
2. Apply moving average and regression forecast.  
3. Write 3 lines on what the results imply for business decisions.
